In [1]:
#importing libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib
from flask import Flask, request, jsonify, send_from_directory
import os
import logging
import warnings

In [2]:
# Suppress warnings
warnings.simplefilter('ignore')

In [3]:
# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [4]:
# Load energy data
data = pd.read_csv('energy_data.csv')

In [5]:
# Convert date column to datetime format
data['date'] = pd.to_datetime(data['date'])

In [6]:
# Extract relevant features from date column
data['day'] = data['date'].dt.day
data['month'] = data['date'].dt.month
data['year'] = data['date'].dt.year
data['day_of_week'] = data['date'].dt.dayofweek
data['is_weekend'] = data['day_of_week'] >= 5

In [7]:
# Drop original date column
data.drop('date', axis=1, inplace=True)

In [8]:
# Preprocess data
X = data.drop(['energy_consumption'], axis=1)
print(X.columns.tolist())  # Add this line to check the expected order
y = data['energy_consumption']

['temperature', 'humidity', 'day', 'month', 'year', 'day_of_week', 'is_weekend']


In [9]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
# Train model
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [11]:
# Save model
model_filename = 'energy_model.pkl'
joblib.dump(model, model_filename)
logger.info(f'Model saved to {model_filename}')

INFO:__main__:Model saved to energy_model.pkl


In [12]:
# Make predictions
y_pred = model.predict(X_test)

In [13]:
# Evaluate model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
logger.info(f'MSE: {mse}, MAE: {mae}, R²: {r2}')

INFO:__main__:MSE: 194.70750000000007, MAE: 11.183333333333332, R²: 0.831503125


In [14]:
# Define Flask API
app = Flask(__name__)

In [15]:
@app.route('/')
def home():
    return 'Energy Consumption Prediction API'

In [16]:
@app.route('/favicon.ico')
def favicon():
    return send_from_directory(os.path.join(app.root_path, 'static'), 'favicon.ico', mimetype='image/vnd.microsoft.icon')

In [17]:
@app.route('/predict', methods=['POST'])
def predict():
    logger.info('Received prediction request')
    data = request.get_json()
    
    # Validate input data
    if not data:
        logger.error('No input data provided')
        return jsonify({'error': 'No input data provided'}), 400
    
    try:
        input_data = pd.DataFrame(data)
        required_columns = ['day', 'month', 'year', 'day_of_week', 'is_weekend']
        if not all(col in input_data.columns for col in required_columns):
            logger.error('Missing required columns')
            return jsonify({'error': 'Missing required columns'}), 400
        
        # Make prediction
        prediction = model.predict(input_data)
        logger.info('Prediction successful')
        return jsonify({'prediction': prediction.tolist()})
    
    except Exception as e:
        logger.error(f'Error during prediction: {str(e)}')
        return jsonify({'error': str(e)}), 500

In [ ]:
# Run Flask API
if __name__ == '__main__':
    from waitress import serve
    # Load the model
    model = joblib.load(model_filename)
    logger.info('Model loaded successfully')
    # Run the app with waitress
    serve(app, host='0.0.0.0', port=5000)

INFO:__main__:Model loaded successfully
INFO:waitress:Serving on http://0.0.0.0:5000
